# converting to images

In [1]:
!pip install pdf2image

In [2]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 1s (227 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
from pdf2image import convert_from_path
import os
# PDF file paths in Colab
pdfPMBOK = "/content/PMBOK6-2017.pdf"  # Make sure to upload your PDF file to Colab
output_folder_1 = "/content/PMBOOKimgs"
if not os.path.exists(output_folder_1):
    os.makedirs(output_folder_1)
def convert_pdf_to_images(pdf_file_path, output_folder, start_page, end_page):
    images = convert_from_path(pdf_file_path, output_folder=output_folder, fmt='png', first_page=start_page, last_page=end_page, output_file="page")
    print("Done.")

convert_pdf_to_images(pdfPMBOK, output_folder_1, 431, 494)


Done.


# deleting figures in on  image

In [ ]:
import cv2
import numpy as np

def detect_and_remove_rectangles(image_path, min_contour_area=1000):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask with the same size as the image, initialized to white
    mask = np.ones_like(image) * 255

    for contour in contours:
        # Calculate the contour area
        area = cv2.contourArea(contour)

        # If the contour area is greater than the minimum threshold
        if area >= min_contour_area:
            # Approximate the contour as a polygon
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the polygon has 4 vertices, indicating a rectangle
            if len(approx) == 4:
                # Get the bounding box coordinates of the rectangle
                x, y, w, h = cv2.boundingRect(contour)

                # Set the region corresponding to the rectangle to black in the mask
                mask[y:y+h, x:x+w] = 0

    # Use the mask to copy the non-rectangle parts of the image to a new image
    result = cv2.bitwise_and(image, mask)

    return result
# Example usage:
output_image = detect_and_remove_rectangles("PMBOOKimgs/page0001-438.png")
cv2.imwrite("output_image.jpg", output_image)


True

In [ ]:
!pip install easyocr

In [ ]:
import easyocr
from PIL import Image

# Initialize the EasyOCR reader with the desired languages
reader = easyocr.Reader(['en'])

# Load the image from which you want to extract text
image = Image.open('output_image.jpg')

# Perform OCR on the image
results = reader.readtext(image)

# Iterate through the OCR results
for (text, bounding_box, prob) in results:
    print(f'{bounding_box}')
# Close the image
image.close()


Figure 11-3. Plan Risk Management: Data Flow Diagram
The Plan Risk Management process should begin when a project is conceived and should be completed early in the
project It may be necessary to revisit this process later in the project life cycle, for example at a major phase change, or
if the project scope changes significantly; or if a subsequent review of risk management effectiveness determines that
the Project Risk Management process requires modification.
C
11.1.1 PLAN RISK MANAGEMENT: INPUTS
11.1.1.1 PROJECT CHARTER
Described in Section 4.1.3.1. The project charter documents the high-level project description and boundaries, high-
level requirements, and risks.
402
Part 1
Guide
8
Noe


# deleting figures in all the images

In [6]:
import cv2
import numpy as np
import os
def detect_and_remove_rectangles(image_path, min_contour_area=1000):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask with the same size as the image, initialized to white
    mask = np.ones_like(image) * 255

    for contour in contours:
        # Calculate the contour area
        area = cv2.contourArea(contour)

        # If the contour area is greater than the minimum threshold
        if area >= min_contour_area:
            # Approximate the contour as a polygon
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the polygon has 4 vertices, indicating a rectangle
            if len(approx) == 4:
                # Get the bounding box coordinates of the rectangle
                x, y, w, h = cv2.boundingRect(contour)

                # Set the region corresponding to the rectangle to black in the mask
                mask[y:y+h, x:x+w] = 0

    # Use the mask to copy the non-rectangle parts of the image to a new image
    result = cv2.bitwise_and(image, mask)

    return result

# Specify the folder containing the images
folder_path = "PMBOOKimgs"

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Specify the output folder
output_folder = "output_folder_PMBOOKimgs"

# Check if the output folder exists, and if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the folder
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)

    # Call the detect_and_remove_rectangles function
    output_image = detect_and_remove_rectangles(image_path)

    # Define the output file path for each processed image
    output_file = os.path.join(output_folder, image_file)

    # Save the processed image to the output folder
    cv2.imwrite(output_file, output_image)

print("Processing completed. Processed images are saved in the '{}' folder.".format(output_folder))


Processing completed. Processed images are saved in the 'output_folder_PMBOOKimgs' folder.


# sortie mch haka

In [8]:
!pip install easyocr

In [9]:
import easyocr
from PIL import Image
import os
import numpy as np

# Initialize the EasyOCR reader with the desired languages
reader = easyocr.Reader(['en'])

# Specify the folder containing your images
image_folder = 'output_folder_PMBOOKimgs'

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]

# Sort the image files by their filenames (assuming they are named in order)
image_files.sort()

# Create an empty list to store the extracted text
all_text = []

# Iterate through the image files and perform OCR
for image_file in image_files:
    image_path = os.path.join(image_folder, image_file)
    image = Image.open(image_path)

    # Convert PIL Image to NumPy array
    image_np = np.array(image)

    # Perform OCR on the image (passing NumPy array)
    results = reader.readtext(image_np)

    # Extract text from each result and append it to the list
    text_list = [result[1] for result in results]
    all_text.append(' '.join(text_list))

    # Close the image
    image.close()

# Print or use the extracted text as needed
for i, text in enumerate(all_text):
    print(f'{text}')


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteI1 PROJECT RISK MANAGEMENT Project Risk Management includes the processes of conducting risk management planning, identification; analysis, response planning, response implementation, and monitoring risk on a project The objectives of project risk management are to increase the probability and/or impact of positive risks and to decrease the probability and/or impact of negative risks, in order to optimize the chances of project success The Project Risk Management processes are: 11.1 Plan Risk Management-The process of defining how to conduct risk management activities for a project. 11.2 Identify Risks--The process of identifying individual project risks as well as sources of overall project risk, and documenting their characteristics. 11.3 Perform Qualitative Risk Analysis_The process of prioritizing individual project risks for further analysis or action by assessing their probability of occurrence and impa

In [10]:
print(all_text)

['I1 PROJECT RISK MANAGEMENT Project Risk Management includes the processes of conducting risk management planning, identification; analysis, response planning, response implementation, and monitoring risk on a project The objectives of project risk management are to increase the probability and/or impact of positive risks and to decrease the probability and/or impact of negative risks, in order to optimize the chances of project success The Project Risk Management processes are: 11.1 Plan Risk Management-The process of defining how to conduct risk management activities for a project. 11.2 Identify Risks--The process of identifying individual project risks as well as sources of overall project risk, and documenting their characteristics. 11.3 Perform Qualitative Risk Analysis_The process of prioritizing individual project risks for further analysis or action by assessing their probability of occurrence and impact as well as other characteristics: 11.4 Perform Quantitative Risk Analysis

#sortie s7i7a

In [11]:
import easyocr
from PIL import Image
import os
import numpy as np

# Initialize the EasyOCR reader with the desired languages
reader = easyocr.Reader(['en'])

# Specify the folder containing the images
folder_path = 'output_folder_PMBOOKimgs'

# List all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Sort the image files based on their names
image_files.sort()

# Define the image to be skipped
skip_image = 'page0001-432.png'

# Loop through each image file
for image_file in image_files:
    # Check if the current image file matches the one to be skipped
    if image_file == skip_image:
        print(f"Skipping {skip_image}")
        continue  # Skip this image and move to the next one

    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_file)

    # Load the image and convert it to a NumPy array
    image = np.array(Image.open(image_path))

    # Perform OCR on the image
    results = reader.readtext(image)

    # Print the OCR results for the current image
    print(f"OCR results for {image_file}:")
    for (text, bounding_box, prob) in results:
        print(f'{bounding_box}')


OCR results for page0001-431.png:
I1
PROJECT RISK MANAGEMENT
Project Risk Management includes the processes of conducting risk management planning, identification; analysis,
response planning, response implementation, and monitoring risk on a project The objectives of project risk management
are to increase the probability and/or impact of positive risks and to decrease the probability and/or impact of negative
risks, in order to optimize the chances of project success
The Project Risk Management processes are:
11.1 Plan Risk Management-The process of defining how to conduct risk management activities for a project.
11.2 Identify Risks--The process of identifying individual project risks as well as sources of overall project risk,
and documenting their characteristics.
11.3 Perform Qualitative Risk Analysis_The process of prioritizing individual project risks for further analysis
or action by assessing their probability of occurrence and impact as well as other characteristics:
11.4 Pe

In [12]:
import easyocr
from PIL import Image
import os
import numpy as np

# Initialize the EasyOCR reader with the desired languages
reader = easyocr.Reader(['en'])

# Specify the folder containing the images
folder_path = 'output_folder_PMBOOKimgs'

# List all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Sort the image files based on their names
image_files.sort()

# Define the image to be skipped
skip_image = 'page0001-432.png'

# Create a text file to store the OCR results
output_file = 'ocr_results_PMBOOKimgs.txt'

# Loop through each image file
with open(output_file, 'w', encoding='utf-8') as txt_file:
    for image_file in image_files:
        # Check if the current image file matches the one to be skipped
        if image_file == skip_image:
            print(f"Skipping {skip_image}")
            continue  # Skip this image and move to the next one

        # Construct the full path to the image
        image_path = os.path.join(folder_path, image_file)

        # Load the image and convert it to a NumPy array
        image = np.array(Image.open(image_path))

        # Perform OCR on the image
        results = reader.readtext(image)

        # Write the OCR results for the current image to the text file
        for (text, bounding_box, prob) in results:
            txt_file.write(f"{bounding_box}\n")

print(f"OCR results have been saved to {output_file}")


Skipping page0001-432.png
OCR results have been saved to ocr_results_PMBOOKimgs.txt


#make a dataframe

In [42]:
import re
import pandas as pd

def extract_concepts_and_definitions_from_text(text_file):
    with open(text_file, 'r', encoding='utf-8') as file:
        text = file.read()

    concepts_and_definitions = []
    current_concept = ""
    current_concept_name = ""
    current_definition = ""

    lines = text.split('\n')

    for line in lines:
        # Check if the line matches the pattern for a concept
        if re.match(r'(\d+(\.\d+)*)\s+(.+)', line):
            if current_concept:
                # Append the current concept and definition to the list as a tuple
                concepts_and_definitions.append((current_concept, current_concept_name, current_definition))

            # Split the line into concept and definition
            concept_match = re.match(r'(\d+(\.\d+)*)\s+(.+)', line)
            current_concept = concept_match.group(1)
            current_concept_name = concept_match.group(3)
            current_definition = ""
        else:
            # If the line does not match the pattern, consider it part of the definition
            current_definition += "\n" + line

    # Append the last concept and definition to the list
    concepts_and_definitions.append((current_concept, current_concept_name, current_definition))

    # Create a DataFrame
    df = pd.DataFrame(concepts_and_definitions, columns=["Number", "Title", "Description"])
    return df

# Example of usage
text_file_path = 'ocr_results_PMBOOKimgs.txt'  # Replace with the path to your text file
df = extract_concepts_and_definitions_from_text(text_file_path)

# Print or process the DataFrame
print(df)


       Number                                  Title  \
0        11.1                   Plan Risk Management   
1        11.2                         Identify Risks   
2        11.3      Perform Qualitative Risk Analysis   
3        11.4     Perform Quantitative Risk Analysis   
4        11.5                    Plan Risk Responses   
..        ...                                    ...   
113  11.7.3.1           WORK PERFORMANCE INFORMATION   
114  11.7.3.2                        CHANGE REQUESTS   
115  11.7.3.3        PROJECT MANAGEMENT PLAN UPDATES   
116  11.7.3.4              PROJECT DOCUMENTS UPDATES   
117  11.7.3.5  ORGANIZATIONAL PROCESS ASSETS UPDATES   

                                           Description  
0    \nThe process of defining how to conduct risk ...  
1    \nThe process of identifying individual projec...  
2    \nThe process of prioritizing individual proje...  
3    \nThe process ofnumerically analyzing the comb...  
4    \nThe process of developing options, 

In [43]:
df

,Number,Title,Description
0,11.1,Plan Risk Management,\nThe process of defining how to conduct risk ...
1,11.2,Identify Risks,\nThe process of identifying individual projec...
2,11.3,Perform Qualitative Risk Analysis,\nThe process of prioritizing individual proje...
3,11.4,Perform Quantitative Risk Analysis,\nThe process ofnumerically analyzing the comb...
4,11.5,Plan Risk Responses,"\nThe process of developing options, selecting..."
...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,\nDescribed in Section 4.5.1.3. Work performan...
114,11.7.3.2,CHANGE REQUESTS,\nY\nDescribed in Section 4.3.3.4. The Monitor...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nProject documents that may be updated as a r...


In [71]:
file_path = 'PMBOOKimgs_data.csv'

df.to_csv(file_path, index=False)

# cleaning

In [44]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    return text
# Apply text cleaning to the "Definition" column
df["Description"] = df["Description"].apply(clean_text)
# Print the cleaned DataFrame
df

,Number,Title,Description
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...
1,11.2,Identify Risks,\nthe process of identifying individual projec...
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting..."
...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,\ndescribed in section 4.5.1.3. work performan...
114,11.7.3.2,CHANGE REQUESTS,\ny\ndescribed in section 4.3.3.4. the monitor...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...


In [45]:
# Function to extract "Described In" values
def extract_described_in(definition):
    described_in_matches = re.findall(r'(?:\(?)described in section (\d+(\.\d+)*)', definition)
    if described_in_matches:
        return ', '.join([match[0] for match in described_in_matches])
    else:
        return None
# Apply the function to the DataFrame
df['Described In'] = df['Description'].apply(extract_described_in)

# Display the DataFrame
df

,Number,Title,Description,Described In
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None
...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,\ndescribed in section 4.5.1.3. work performan...,4.5.1.3
114,11.7.3.2,CHANGE REQUESTS,\ny\ndescribed in section 4.3.3.4. the monitor...,4.3.3.4
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2"


In [46]:
# Function to remove "described in" part
def remove_described_in(definition):
    return re.sub(r'\(?\s*described in section \d+(\.\d+)*\s*\)?', '', definition)
# Apply the function to the "Definition" column
df['Description'] = df['Description'].apply(remove_described_in)
# Display the DataFrame
df

,Number,Title,Description,Described In
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None
...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2"


In [1]:
def extract_figures(definition):
    figure_matches = re.findall(r'figure(\d{1,2}-\d{1,2}[a-zA-Z]{0,6})', definition)
    if figure_matches:
        figures = ', '.join(figure_matches)
        return figures
    else:
        return None
# Apply the function to extract figures and add them to the "Figure" column
df['Figure'] = df['Description'].apply(extract_figures)

# Display the DataFrame
df

NameError: name 'df' is not defined

In [48]:
import pandas as pd

# Filter the original DataFrame to get rows with non-None values in the "Figure" column
non_none_figures_df = df[df['Figure'].notna()]

# Display the resulting DataFrame
non_none_figures_df

,Number,Title,Description,Described In,Figure
6,11.7,Monitor Risks,\nthe process of monitoring the implementation...,None,11-1
7,11.1,PLAN RISK MANAGEMENT,\nplan risk management is the process of defin...,None,"11-3, 11-2, 11-3"
19,11.1.3.1,RISK MANAGEMENT PLAN,\nthe risk management plan is a component of t...,11.3.2.6,"11-4, 11-5, 11-5"
20,11.2,IDENTIFY RISKS,\nidentify risks is the process of identifying...,None,"11-6, 11-7, 11-7"
22,11.2.1.1,PROJECT MANAGEMENT PLAN,. project management plan components include b...,"4.2.3.1, 6.1.3.1, 7.1.3.1, 8.1.3.1, 9.1.3.1, 1...",11-4
39,11.3,PERFORM QUALITATIVE RISK ANALYSIS,\nperform qualitative risk analysis is the pro...,None,"11-8, 11-9, 11-8, 11-9"
50,11.3.2.5,RISK CATEGORIZATION,\nrisks to the project can be categorized by s...,None,11-4
51,11.3.2.6,DATA REPRESENTATION,\ndata representation techniques that can be u...,None,"11-5, 11-10, 11-10"
55,11.4,PERFORM QUANTITATIVE RISK ANALYSIS,\nperform quantitative risk analysis is the pr...,None,"11-11, 11-12, 11-11, 11-12"
66,11.4.2.5,DATA ANALYSIS,\ndata analysis techniques that can be used du...,None,"11-13, 11-13, 11-14, 11-14, 11-15, 11-15"


In [49]:
def clean_text(text):
    # Remove special characters and symbols (retain letters, numbers, and basic punctuation)
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra spaces and trim leading/trailing spaces
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text.lower()
df['Definition'] = df['Description'].apply(clean_text)

In [50]:
df

,Number,Title,Description,Described In,Figure,Definition
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,the process of defining how to conduct risk ma...
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,the process of identifying individual project ...
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,the process of prioritizing individual project...
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,the process ofnumerically analyzing the combin...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,the process of developing options selecting st...
...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,work performance information includes informat...
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,y the monitor risks process may result in a ch...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,change to the project management plan goes thr...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,project documents that may be updated as a res...


In [51]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [52]:
from nltk.tokenize import word_tokenize
def tokenize(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)
    return tokens
df['Definition'] = df['Description'].apply(tokenize)

In [53]:
df

,Number,Title,Description,Described In,Figure,Definition
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,"[the, process, of, defining, how, to, conduct,..."
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,"[the, process, of, identifying, individual, pr..."
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,"[the, process, of, prioritizing, individual, p..."
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,"[the, process, ofnumerically, analyzing, the, ..."
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,"[the, process, of, developing, options, ,, sel..."
...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,"[., work, performance, information, includes, ..."
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,"[y., the, monitor, risks, process, may, result..."
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,"[change, to, the, project, management, plan, g..."
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,"[project, documents, that, may, be, updated, a..."


In [54]:
import pandas as pd
from nltk import word_tokenize, pos_tag

In [55]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [56]:
def pos_tagging(text):
    # Tokenize the text into words
    words = tokenize(text)

    # Perform POS tagging
    pos_tags = pos_tag(words)

    return pos_tags
df['Definition'] = df['Description'].apply(pos_tagging)

In [57]:
df

,Number,Title,Description,Described In,Figure,Definition
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,"[(the, DT), (process, NN), (of, IN), (defining..."
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,"[(the, DT), (process, NN), (of, IN), (identify..."
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,"[(the, DT), (process, NN), (of, IN), (prioriti..."
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,"[(the, DT), (process, NN), (ofnumerically, RB)..."
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,"[(the, DT), (process, NN), (of, IN), (developi..."
...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,"[(., .), (work, NN), (performance, NN), (infor..."
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,"[(y., VB), (the, DT), (monitor, NN), (risks, N..."
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,"[(change, NN), (to, TO), (the, DT), (project, ..."
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,"[(project, NN), (documents, NNS), (that, WDT),..."


In [58]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [59]:
from nltk.corpus import stopwords

def remove_stopwords_pos(pos_tags):
    # Define the parts of speech tags you want to keep
    allowed_pos_tags = ['N', 'V', 'R', 'J']  # Noun, Verb, Adverb, Adjective

    # Create a list of stopwords based on the specified POS tags
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word, pos in pos_tags if pos[0] in allowed_pos_tags and word.lower() not in stop_words]

    return filtered_words

df['Definition'] = df['Definition'].apply(remove_stopwords_pos)


In [60]:
df

,Number,Title,Description,Described In,Figure,Definition
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,"[process, defining, conduct, risk, management,..."
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,"[process, identifying, individual, project, ri..."
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,"[process, prioritizing, individual, project, r..."
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,"[process, ofnumerically, analyzing, combined, ..."
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,"[process, developing, options, selecting, stra..."
...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,"[work, performance, information, includes, inf..."
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,"[y., monitor, risks, process, result, change, ..."
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,"[change, project, management, plan, goes, orga..."
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,"[project, documents, updated, result, carrying..."


In [61]:
def tokens_to_text(tokens):
    cleaned_text = ' '.join(tokens)
    return cleaned_text
df['Definition'] = df['Definition'].apply(tokens_to_text)

In [62]:
df

,Number,Title,Description,Described In,Figure,Definition
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,process defining conduct risk management activ...
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,process identifying individual project risks w...
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,process prioritizing individual project risks ...
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,process ofnumerically analyzing combined effec...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,process developing options selecting strategie...
...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,work performance information includes informat...
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,y. monitor risks process result change request...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,change project management plan goes organizati...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,project documents updated result carrying proc...


In [31]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [32]:
from nltk.stem import WordNetLemmatizer
# Create a lemmatizer object
lemmatizer = WordNetLemmatizer()

# Function to lemmatize a text
def lemmatize_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(lemmatized_tokens)

# Apply lemmatization to the 'Subject' column
df['Definition'] = df['Definition'].apply(lemmatize_text)

In [33]:
df

,Number,Title,Description,Described In,Figure,Definition
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,process defining conduct risk management activ...
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,process identifying individual project risk we...
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,process prioritizing individual project risk a...
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,process ofnumerically analyzing combined effec...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,process developing option selecting strategy a...
...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,work performance information includes informat...
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,y. monitor risk process result change request ...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,change project management plan go organization...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,project document updated result carrying proce...


In [64]:
!pip install textblob
import nltk
nltk.download('brown')
nltk.download('punkt')
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [65]:
import pandas as pd
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")
# Define a function to lemmatize text using spaCy
def lemmatize_text(text):
    doc = nlp(text)
    return ' '.join([token.lemma_ for token in doc])

# Apply lemmatization to the "Text" column
df['Lemmatized_Text'] = df['Definition'].apply(lemmatize_text)

In [66]:
df

,Number,Title,Description,Described In,Figure,Definition,Lemmatized_Text
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,process defining conduct risk management activ...,process define conduct risk management activit...
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,process identifying individual project risks w...,process identify individual project risk well ...
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,process prioritizing individual project risks ...,process prioritize individual project risk ana...
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,process ofnumerically analyzing combined effec...,process ofnumerically analyze combine effect i...
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,process developing options selecting strategie...,process develop option select strategy agree a...
...,...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,work performance information includes informat...,work performance information include informati...
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,y. monitor risks process result change request...,y. monitor risks process result change request...
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,change project management plan goes organizati...,change project management plan go organization...
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,project documents updated result carrying proc...,project document update result carry process i...


In [37]:
import networkx as nx
import matplotlib.pyplot as plt
def chunk_words_by_meaning(text):
    # Tokenize the text into words
    words = tokenize(text)

    # Perform POS tagging
    pos_tags = pos_tagging(text)
    chunks = []
    # Define a grammar for chunking (NP: Noun Phrase)
    # Define a grammar for chunking
    grammar = r"""
        NP: {<JJ.*>*<PRP|NN.*>+}   # chunk determiners, adjectives, personal pronouns, and nouns
        VP: {<MD>*<VB.*>+}      # chunk verb phrases with optional NP
        CHUNK: {<NP><VP>+<NP>}  # chunk relation with NP-VP-NP pattern
    """

    # Create a chunk parser with the defined grammar

    chunk_parser = RegexpParser(grammar)


    # Apply chunking to POS-tagged words
    tree = chunk_parser.parse(pos_tags)

        # Extract phrases from the tree
    for subtree in tree.subtrees():
        if subtree.label() == 'CHUNK':
            listt=[]
            for sub in subtree:
                NP = ' '.join([token for token, _ in sub.leaves()])
                listt.append(NP)
            chunks.append(listt)
    print(chunks)

    return chunks,tree


In [38]:
new_df=pd.DataFrame(columns=["Title","Description","Subject","Relation","Object"])

In [40]:
from nltk import RegexpParser

In [41]:
for index, row in df.iterrows():
    text = row["Definition"]
    chunks,tree = chunk_words_by_meaning(text)
    for k in chunks:
        new_df.at[index, 'Title'] = row["Title"]
        new_df.at[index, 'Description'] = row["Description"]
        new_df.at[index, 'Subject']=k[0]
        new_df.at[index, 'Relation']=k[1]
        new_df.at[index, 'Object']=k[2]

[['process', 'defining', 'conduct risk management activity project']]
[['process', 'identifying', 'individual project risk']]
[['process', 'prioritizing', 'individual project risk analysis action']]
[['effect', 'identified', 'individual project risk source uncertainty']]
[['process', 'developing', 'option'], ['strategy', 'agreeing', 'action address']]
[['responsesthe process', 'implementing', 'agreed-upon risk response plan']]
[['process', 'monitoring', 'implementation agreed-upon risk response plan'], ['risk', 'identifying analyzing', 'new risk'], ['overview project risk management process project management risk process', 'presented', 'discrete process'], ['degree complexity aim deliver', 'benefit', 'context constraint assumption'], ['stakeholder expectation', 'conflicting changing', 'organization choose'], ['project risk', 'controlled', 'intentional manner order create value'], ['risk reward project risk management aim identify manage risk', 'addressed', 'project management process'

In [192]:
new_df

,Title,Description,Subject,Relation,Object
1,Identify Risks,\nthe process of identifying individual projec...,process,identify,individual project risk
2,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,process,prioritize,individual project risk analysis action assess...
3,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,analyze combine effect,identify,individual project risk source uncertainty
4,Plan Risk Responses,"\nthe process of developing options, selecting...",process,develop,option select strategy
6,Monitor Risks,\nthe process of monitoring the implementation...,work reprioritize project progress base,improved,understand current risk exposure part guide di...
...,...,...,...,...,...
113,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,individual risk,occur,expectation
114,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,integrate change control process section chang...,include recommend,corrective preventive action address
115,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,change project management plan,go,organization change control process change req...
116,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,available monitor risk process risk report,update,reflect current status


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'Definition' column
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Definition'])

# Get the feature names (words) from the TF-IDF vectorizer
feature_names = tfidf_vectorizer.get_feature_names_out()

# Find the top N keywords for each row in the DataFrame
top_keywords = []
num_top_keywords = 5  # You can adjust this value as per your needs

for row in tfidf_matrix:
    sorted_indices = row.indices[np.argsort(row.data)[::-1][:num_top_keywords]]
    keywords = [feature_names[idx] for idx in sorted_indices]
    top_keywords.append(keywords)

# Add the 'Keywords' column to the DataFrame
df['Keywords'] = top_keywords


In [70]:
df

,Number,Title,Description,Described In,Figure,Definition,Lemmatized_Text,Keywords
0,11.1,Plan Risk Management,\nthe process of defining how to conduct risk ...,None,None,process defining conduct risk management activ...,process define conduct risk management activit...,"[defining, conduct, activities, management, pr..."
1,11.2,Identify Risks,\nthe process of identifying individual projec...,None,None,process identifying individual project risks w...,process identify individual project risk well ...,"[documenting, characteristics, identifying, pr..."
2,11.3,Perform Qualitative Risk Analysis,\nthe process of prioritizing individual proje...,None,None,process prioritizing individual project risks ...,process prioritize individual project risk ana...,"[prioritizing, assessing, characteristics, occ..."
3,11.4,Perform Quantitative Risk Analysis,\nthe process ofnumerically analyzing the comb...,None,None,process ofnumerically analyzing combined effec...,process ofnumerically analyze combine effect i...,"[ofnumerically, combined, analyzing, effect, u..."
4,11.5,Plan Risk Responses,"\nthe process of developing options, selecting...",None,None,process developing options selecting strategie...,process develop option select strategy agree a...,"[agreeing, treat, options, developing, selecting]"
...,...,...,...,...,...,...,...,...
113,11.7.3.1,WORK PERFORMANCE INFORMATION,. work performance information includes inform...,4.5.1.3,None,work performance information includes informat...,work performance information include informati...,"[information, expectation, comparing, performi..."
114,11.7.3.2,CHANGE REQUESTS,\ny. the monitor risks process may result in a...,4.3.3.4,None,y. monitor risks process result change request...,y. monitor risks process result change request...,"[change, requests, address, corrective, recomm..."
115,11.7.3.3,PROJECT MANAGEMENT PLAN UPDATES,\nchange to the project management plan goes t...,None,None,change project management plan goes organizati...,change project management plan go organization...,"[change, goes, component, affect, plan]"
116,11.7.3.4,PROJECT DOCUMENTS UPDATES,\nproject documents that may be updated as a r...,"4.1.3.2, 4.3.3.3, 4.4.3.1, 11.2.3.1, 11.2.3.2",None,project documents updated result carrying proc...,project document update result carry process i...,"[new, updated, risks, monitor, risk]"


#practice standard

In [ ]:
from pdf2image import convert_from_path

# PDF file paths in Colab
pdfPMI = "/content/practice-standard-project-risk-management.pdf"
output_folder_2="/content/Practiceimgs"
if not os.path.exists(output_folder_2):
    os.makedirs(output_folder_2)
def convert_pdf_to_images(pdf_file_path, output_folder, start_page, end_page):
    images = convert_from_path(pdf_file_path, output_folder=output_folder, fmt='png', first_page=start_page, last_page=end_page, output_file="page")
    print("Done.")

convert_pdf_to_images(pdfPMI, output_folder_2, 1, 116)

Done.


In [ ]:
import cv2
import numpy as np
import os
def detect_and_remove_rectangles(image_path, min_contour_area=1000):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask with the same size as the image, initialized to white
    mask = np.ones_like(image) * 255

    for contour in contours:
        # Calculate the contour area
        area = cv2.contourArea(contour)

        # If the contour area is greater than the minimum threshold
        if area >= min_contour_area:
            # Approximate the contour as a polygon
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the polygon has 4 vertices, indicating a rectangle
            if len(approx) == 4:
                # Get the bounding box coordinates of the rectangle
                x, y, w, h = cv2.boundingRect(contour)

                # Set the region corresponding to the rectangle to black in the mask
                mask[y:y+h, x:x+w] = 0

    # Use the mask to copy the non-rectangle parts of the image to a new image
    result = cv2.bitwise_and(image, mask)

    return result

# Specify the folder containing the images
folder_path = "Practiceimgs"

# Get a list of all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Specify the output folder
output_folder = "output_folder_Practiceimgs"

# Check if the output folder exists, and if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the folder
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)

    # Call the detect_and_remove_rectangles function
    output_image = detect_and_remove_rectangles(image_path)

    # Define the output file path for each processed image
    output_file = os.path.join(output_folder, image_file)

    # Save the processed image to the output folder
    cv2.imwrite(output_file, output_image)

print("Processing completed. Processed images are saved in the '{}' folder.".format(output_folder))


Processing completed. Processed images are saved in the 'output_folder_Practiceimgs' folder.


In [ ]:
import easyocr
from PIL import Image
import os
import numpy as np

# Initialize the EasyOCR reader with the desired languages
reader = easyocr.Reader(['en'])

# Specify the folder containing the images
folder_path = 'output_folder_Practiceimgs'

# List all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Sort the image files based on their names
image_files.sort()

# Create a text file to store the OCR results
output_file = 'ocr_results_Practiceimgs.txt'

# Loop through each image file
with open(output_file, 'w', encoding='utf-8') as txt_file:
    for image_file in image_files:

        # Construct the full path to the image
        image_path = os.path.join(folder_path, image_file)

        # Load the image and convert it to a NumPy array
        image = np.array(Image.open(image_path))

        # Perform OCR on the image
        results = reader.readtext(image)

        # Write the OCR results for the current image to the text file
        for (text, bounding_box, prob) in results:
            txt_file.write(f"{bounding_box}\n")

print(f"OCR results have been saved to {output_file}")


OCR results have been saved to ocr_results_Practiceimgs.txt


In [ ]:
import re
import pandas as pd

def extract_concepts_and_definitions_from_text(text_file):
    with open(text_file, 'r', encoding='utf-8') as file:
        text = file.read()

    concepts_and_definitions = []
    current_concept = ""
    current_concept_name = ""
    current_definition = ""

    lines = text.split('\n')

    for line in lines:
        # Check if the line matches the pattern for a concept
        if re.match(r'(\d+(\.\d+)*)\s+(.+)', line):
            if current_concept:
                # Append the current concept and definition to the list as a tuple
                concepts_and_definitions.append((current_concept, current_concept_name, current_definition))

            # Split the line into concept and definition
            concept_match = re.match(r'(\d+(\.\d+)*)\s+(.+)', line)
            current_concept = concept_match.group(1)
            current_concept_name = concept_match.group(3)
            current_definition = ""
        else:
            # If the line does not match the pattern, consider it part of the definition
            current_definition += "\n" + line

    # Append the last concept and definition to the list
    concepts_and_definitions.append((current_concept, current_concept_name, current_definition))

    # Create a DataFrame
    df = pd.DataFrame(concepts_and_definitions, columns=["Number", "Concept", "Definition"])
    return df

# Example of usage
text_file_path = 'ocr_results_Practiceimgs.txt'  # Replace with the path to your text file
df = extract_concepts_and_definitions_from_text(text_file_path)

# Print or process the DataFrame
print(df)


     Number                                            Concept  \
0        10                                  9 8 7 6 5 4 3 2 1   
1     02009  Project Management Institute. Practice Standar...   
2    5.2.10                                        Objectivity   
3     02009  Project Management Institute. Practice Standar...   
4     02009  Project Management Institute. Practice Standar...   
..      ...                                                ...   
149   02009  Project Management Institute. Practice Standar...   
150   02009  Project Management Institute. Practice Standar...   
151   02009  Project Management Institute. Practice Standar...   
152   02009  Project Management Institute. Practice Standar...   
153   02009  Project Management Institute. Practice Standar...   

                                            Definition  
0    \nNOTICE\nThe Project Management Institute, In...  
1    \nTABLE OF CONTENTS\nCHAPTER 5\nIDENTIFY RISKS...  
2    \n27\n5.3\nTools and Techniques

In [ ]:
df

,Number,Concept,Definition
0,10,9 8 7 6 5 4 3 2 1,"\nNOTICE\nThe Project Management Institute, In..."
1,02009,Project Management Institute. Practice Standar...,\nTABLE OF CONTENTS\nCHAPTER 5\nIDENTIFY RISKS...
2,5.2.10,Objectivity,\n27\n5.3\nTools and Techniques for the Identi...
3,02009,Project Management Institute. Practice Standar...,\nTABLE OF CONTENTS\nCHAPTER 7\nPERFORM QUANTI...
4,02009,Project Management Institute. Practice Standar...,\nTABLE OF CONTENTS\n8.4\nTools and Techniques...
...,...,...,...
149,02009,Project Management Institute. Practice Standar...,\nthey\nAPPENDIX D\nNo template is offered for...
150,02009,Project Management Institute. Practice Standar...,\nAPPENDIX D
151,02009,Project Management Institute. Practice Standar...,\nAPPENDIX D\nD.6.1.1\nReserve Analysis\nAn an...
152,02009,Project Management Institute. Practice Standar...,\ngiven\nAPPENDIX D\nD.6.1.2\nRisk Audits\nRis...


In [ ]:
file_path = 'Practiceimgs_data.csv'

df.to_csv(file_path, index=False)

In [ ]:
# Specify the file path
file_path = 'ocr_results_PMBOOKimgs.txt'

# Open the file in read mode
with open(file_path, 'r') as file:
    text = file.read()

# Print the content of the file
print(text)


I1
PROJECT RISK MANAGEMENT
Project Risk Management includes the processes of conducting risk management planning, identification; analysis,
response planning, response implementation, and monitoring risk on a project The objectives of project risk management
are to increase the probability and/or impact of positive risks and to decrease the probability and/or impact of negative
risks, in order to optimize the chances of project success
The Project Risk Management processes are:
11.1 Plan Risk Management
The process of defining how to conduct risk management activities for a project.
11.2 Identify Risks
The process of identifying individual project risks as well as sources of overall project risk,
and documenting their characteristics.
11.3 Perform Qualitative Risk Analysis
The process of prioritizing individual project risks for further analysis
or action by assessing their probability of occurrence and impact as well as other characteristics:
11.4 Perform Quantitative Risk Analysis
Th

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Sample text (replace with your extracted text)

# Tokenization: Split the text into words
words = word_tokenize(text)

# Removing stop words: Remove common words like "the" and "is"
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in words if word.lower() not in stop_words]

# Lemmatization: Reduce words to their root forms
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

# The result will be a list of preprocessed words
print(lemmatized_words)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['I1', 'PROJECT', 'RISK', 'MANAGEMENT', 'Project', 'Risk', 'Management', 'includes', 'process', 'conducting', 'risk', 'management', 'planning', ',', 'identification', ';', 'analysis', ',', 'response', 'planning', ',', 'response', 'implementation', ',', 'monitoring', 'risk', 'project', 'objective', 'project', 'risk', 'management', 'increase', 'probability', 'and/or', 'impact', 'positive', 'risk', 'decrease', 'probability', 'and/or', 'impact', 'negative', 'risk', ',', 'order', 'optimize', 'chance', 'project', 'success', 'Project', 'Risk', 'Management', 'process', ':', '11.1', 'Plan', 'Risk', 'Management', 'process', 'defining', 'conduct', 'risk', 'management', 'activity', 'project', '.', '11.2', 'Identify', 'Risks', 'process', 'identifying', 'individual', 'project', 'risk', 'well', 'source', 'overall', 'project', 'risk', ',', 'documenting', 'characteristic', '.', '11.3', 'Perform', 'Qualitative', 'Risk', 'Analysis', 'process', 'prioritizing', 'individual', 'project', 'risk', 'analysis', 

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Sample preprocessed text (replace with your actual preprocessed text)

# Function to extract relevant information
def extract_risk_management_info(text):
    sentences = sent_tokenize(text)
    risk_info = {
        "Risk Identification": [],
        "Risk Assessment": [],
        "Risk Response": []
    }

    for sentence in sentences:
        for section in risk_info.keys():
            if section in sentence:
                tokens = word_tokenize(sentence)
                filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
                risk_info[section].append(" ".join(filtered_tokens))

    return risk_info

# Extract relevant information
extracted_info = extract_risk_management_info(text)

# Print the extracted information
for section, data in extracted_info.items():
    print(f"{section}:\n{data}\n")


Risk Identification:
[]

Risk Assessment:
[]

Risk Response:
['perform qualitative risk analysis process prioritizing individual project risks analysis action assessing probability occurrence impact well characteristics perform quantitative risk analysis process ofnumerically analyzing combined effect identified individual project risks sources uncertainty overall project objectives plan risk responses process developing options selecting strategies agreeing actions address overall project risk exposure well treat individual project risks implement risk responsesthe process implementing risk response plans monitor risks process monitoring implementation risk response plans tracking identified risks identifying analyzing new risks evaluating risk process effectiveness throughout project figure provides overview project risk management processes', 'preliminary risk responses may also identified recorded reviewed confirmed part plan risk responses process identify risks iterative process 

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Sample preprocessed text (replace with your actual preprocessed text)
preprocessed_text = """
Chapter 11: Risk Management
11.1 Introduction
This chapter discusses the principles and guidelines for risk management in project management.

11.2 Risk Identification
Risk identification is a crucial step in the risk management process. There are various methods for identifying risks, including brainstorming, expert interviews, and historical data analysis.

11.3 Risk Assessment
Risk assessment criteria should be established to evaluate the impact and likelihood of identified risks. Common criteria include probability, severity, and project impact.

11.4 Risk Response
Once risks are identified and assessed, risk response strategies should be developed. These strategies may include risk mitigation, risk avoidance, risk transfer, or risk acceptance.

"""

# Function to extract relevant information
def extract_risk_management_info(text):
    sentences = sent_tokenize(text)
    risk_info = {
        "Risk Identification": [],
        "Risk Assessment": [],
        "Risk Response": []
    }

    for sentence in sentences:
        for section in risk_info.keys():
            if section in sentence:
                tokens = word_tokenize(sentence)
                filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stopwords.words('english')]
                risk_info[section].append(" ".join(filtered_tokens))

    return risk_info

# Extract relevant information
extracted_info = extract_risk_management_info(preprocessed_text)

# Print the extracted information
for section, data in extracted_info.items():
    print(f"{section}:\n{data}\n")


Risk Identification:
['risk identification risk identification crucial step risk management process']

Risk Assessment:
['risk assessment risk assessment criteria established evaluate impact likelihood identified risks']

Risk Response:
['risk response risks identified assessed risk response strategies developed']

